In [1]:
#正则表达式的学习
import re
match = re.search(r'[1-9]\d{5}', 'BIT 100081')
if match:
    print(match.group(0))
print(type(match))

100081
<class '_sre.SRE_Match'>


In [2]:
#Match对象的几个属性和方法的使用
import re
m = re.search(r'[1-9]\d{5}', 'BIT100081 TSU100084')
print(m.string)
print(m.re)
print(m.pos)
print(m.endpos)

#方法的使用
print(m.group(0))
print(m.start())
print(m.end())
print(m.span())

BIT100081 TSU100084
re.compile('[1-9]\\d{5}')
0
19
100081
3
9
(3, 9)


In [3]:
import re
#search函数的使用
match = re.search(r'[1-9]\d{5}', 'BIT 100081')
if match:
    print(match.group(0))
#match函数的使用
match = re.match(r'[1-9]\d{5}', '100081 BIT')
if match:
    print(match.group(0))
#findall函数的使用
match = re.findall(r'[1-9]\d{5}', 'BIT100081 TSU100084')
if match:
    print(match)
#split函数的使用
match = re.split(r'[1-9]\d{5}', 'BIT100081 TSU100084')
print(match)
match = re.split(r'[1-9]\d{5}', 'BIT100081 TSU100084', maxsplit=1)
print(match)
#finditer函数的使用
for m in re.finditer(r'[1-9]\d{5}', 'BIT100081 TSU100084'):
    if m:
        print(m.group(0))
#sub函数的使用
match = re.sub(r'[1-9]\d{5}', ':zipcode', 'BIT100081 TSU100084')
print(match)

100081
100081
['100081', '100084']
['BIT', ' TSU', '']
['BIT', ' TSU100084']
100081
100084
BIT:zipcode TSU:zipcode


In [1]:
#淘宝商品比价定向爬虫
import requests
import re

def getHTMLText(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""

def parsePage(ilt, html):
    try:
        plt = re.findall(r'\"view_price\"\:\"[\d\.]*\"', html)
        tlt = re.findall(r'\"raw_title\"\:\".*?\"', html)
        for i in range(len(plt)):
            price = eval(plt[i].split(':')[1])
            title = eval(tlt[i].split(':')[1])
            ilt.append([price, title])
    except:
        print("")

def printGoodsList(ilt):
    tplt = "{:4}\t{:8}\t{:16}"
    print(tplt.format("序号", "价格", "商品名称"))
    count = 0
    for g in ilt:
        count = count + 1
        print(tplt.format(count, g[0], g[1]))
    
def main():
    goods = '书包'
    depth = 2
    start_url = 'https://s.taobao.com/search?q=' + goods
    infoList = []
    for i in range(depth):
        try:
            url = start_url + '&s=' + str(44*i)
            html = getHTMLText(url)
            parsePage(infoList, html)
        except:
            continue
    printGoodsList(infoList)

main()

序号  	价格      	商品名称            


In [ ]:
#股票数据定向爬虫
import requests
from bs4 import BeautifulSoup
import re
import traceback

def getHTMLText(url):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""

def getStockList(lst, stockURL):
    html = getHTMLText(stockURL)
    soup = BeautifulSoup(html, "html.parser")
    a = soup.find_all('a')
    for i in a:
        try:
            href = i.attrs['href']
            lst.append(re.findall(r"[s][hz]\d{6}", href)[0])
        except:
            continue

def getStockInfo(lst, stockURL, fpath):
    for stock in lst:
        url = stockURL + stock + ".html"
        html = getHTMLText(url)
        try:
            if html == "":
                continue
            infoDict = {}
            soup = BeautifulSoup(html, 'html.parser')
            stockInfo = soup.find('div', attrs={'class':'stock-bets'})
            name = stockInfo.find_all(attrs={'class':'bets-name'})[0]
            infoDict.update({'股票名称': name.text.split()[0]})
            
            keyList = stockInfo.find_all('dt')
            valueList = stockInfo.find_all('dd')
            for i in range(len(keyList)):
                key = keyList[i].text
                val = valueList[i].text
                infoDict[key] = val
            with open(fpath, 'a', encoding = 'utf-8') as f:
                f.write(str(infoDict) + '\n')
        except:
            #traceback.print_exc()
            continue
    
def main():
    stock_list_url = 'http://quote.eastmoney.com/stocklist.html'
    stock_info_url = 'http://gupiao.baidu.com/stock/'
    output_file = './spider_BaiduStockInfo.txt'
    slist = []
    getStockList(slist, stock_list_url)
    getStockInfo(slist, stock_info_url, output_file)

main()

In [ ]:
#股票数据定向爬虫实例优化
import requests
from bs4 import BeautifulSoup
import re
import traceback

def getHTMLText(url, code='utf-8'):
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        r.encoding = code
        return r.text
    except:
        return ""

def getStockList(lst, stockURL):
    html = getHTMLText(stockURL, 'GB2312')
    soup = BeautifulSoup(html, "html.parser")
    a = soup.find_all('a')
    for i in a:
        try:
            href = i.attrs['href']
            lst.append(re.findall(r"[s][hz]\d{6}", href)[0])
        except:
            continue

def getStockInfo(lst, stockURL, fpath):
    count = 0
    for stock in lst:
        url = stockURL + stock + ".html"
        html = getHTMLText(url)
        try:
            if html == "":
                continue
            infoDict = {}
            soup = BeautifulSoup(html, 'html.parser')
            stockInfo = soup.find('div', attrs={'class':'stock-bets'})
            name = stockInfo.find_all(attrs={'class':'bets-name'})[0]
            infoDict.update({'股票名称': name.text.split()[0]})
            
            keyList = stockInfo.find_all('dt')
            valueList = stockInfo.find_all('dd')
            for i in range(len(keyList)):
                key = keyList[i].text
                val = valueList[i].text
                infoDict[key] = val
            with open(fpath, 'a', encoding = 'utf-8') as f:
                f.write(str(infoDict) + '\n')
                #增加进度条显示当前爬取的进度
                count = count + 1
                print('\r当前进度：{:.2f}%' .format(count*100/len(lst)), end='')
        except:
            count = count + 1
            print('\r当前进度：{:.2f}%' .format(count*100/len(lst)), end='')
            #traceback.print_exc()
            continue

def main():
    stock_list_url = 'http://quote.eastmoney.com/stocklist.html'
    stock_info_url = 'http://gupiao.baidu.com/stock/'
    output_file = './spider_BaiduStockInfo(1).txt'
    slist = []
    getStockList(slist, stock_list_url)
    getStockInfo(slist, stock_info_url, output_file)

main()

当前进度：15.04%